In [7]:
%pylab inline
from glob import glob
from numpy import load
import pandas as pd
import numpy as np
import copy
from scipy.stats import skew, kurtosis

from time import time
import pickle as pkl
import os
import cv2
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['time', 'copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [8]:
import os
path = os.getcwd()
_uname = path.split('/')[2]
poverty_dir=f'/home/{_uname}/public/cs255-sp22-a00-public/poverty/'
image_dir=poverty_dir+'anon_images/'
image_dir

'/home/ans037/public/cs255-sp22-a00-public/poverty/anon_images/'

In [9]:
train_table=f'/home/{_uname}/public/Datasets_public/Final_Project_Data/train.csv'
df=pd.read_csv(train_table,index_col=0)
df.index=df['filename']

In [10]:
def getImage(image):
    M = np.load(image)
    l = [cv2.resize(M['x'][i, :, :], (20, 20)) for i in range(8)]
    return np.array(l)

In [11]:
df1=pd.read_pickle('df1.pkl')

In [12]:
df1.columns

Index(['country', 'urban', 'label', 'nl_mean', 'MBI_mean', 'MBI_std',
       'MBI_skewness', 'MBI_kurtosis', 'MBI_percentile_25',
       'MBI_percentile_50',
       ...
       'TEMP1_percentile_25', 'TEMP1_percentile_50', 'TEMP1_percentile_75',
       'NL_mean', 'NL_std', 'NL_skewness', 'NL_kurtosis', 'NL_percentile_25',
       'NL_percentile_50', 'NL_percentile_75'],
      dtype='object', length=186)

In [13]:
df['Image'] = df.apply(lambda x: getImage(image_dir + '/'+ x['filename']), axis=1)

In [14]:
df1 = df.copy(deep=True)

In [15]:
def extract_features(band):
    features = {}
    features['mean'] = np.mean(band)
    features['std'] = np.std(band)
    features['skewness'] = skew(band)
    features['kurtosis'] = kurtosis(band)
    features['percentile_25'] = np.percentile(band, 25)
    features['percentile_50'] = np.percentile(band, 50)
    features['percentile_75'] = np.percentile(band, 75)
    return features

def process_outliers(x):
    _min = np.percentile(x, 0.1)
    _max = np.percentile(x,99.9)
        
    x = (x - _min)/(_max - _min)
    x = np.where(x > 1, 1, x)
    x = np.where(x < 0, 0, x)
    return x
    

def calculate_indices(df, train=True):
    if train:
        df = df.drop(['filename', 'wealthpooled'], axis=1)
    else:
        df = df.drop(['filename'], axis=1)
    
    df = df.reset_index(drop=True)
    band_names = ['Red', 'Green', 'Blue', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1', 'NL']
    
    for i in range(len(band_names)):
        df[band_names[i]] = df.apply(lambda x: x['Image'][i, :, :], axis=1)

    # Calculate the bare soil indices
    df['MBI'] = (df['SWIR1'] - df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['SWIR2'] + df['NIR']) + 0.5
    df['BSI'] = (df['SWIR1'] + df['Red'] - df['NIR'] - df['Blue']) / (df['SWIR1'] + df['Red'] + df['NIR'] + df['Blue'])
    df['NDSI1'] = (df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['NIR'])
    df['NDSI2'] = (df['SWIR1'] - df['Green']) / (df['SWIR1'] + df['Green'])
    df['BI'] = df['Red'] + df['SWIR1'] - df['NIR']
    df['DBSI'] = (df['SWIR1'] - df['Green']) / (df['SWIR1'] + df['Green']) - (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])

    df['BAEI'] = (df['Red'] + 0.3) / (df['Green'] + df['SWIR1'])
    df['BUI'] = (df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['NIR']) - (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])
    df['NBI'] = (df['Red'] - df['SWIR1']) / df['NIR']
    df['BRBA'] = df['Red'] / df['SWIR1']
    df['IBI'] = (2 * df['SWIR1'] / (df['SWIR1'] + df['NIR'])) - ((df['NIR'] / (df['NIR'] - df['Red']) + df['Green'] / (df['Green'] + df['SWIR1'])) / 2) * (df['SWIR1'] / (df['SWIR1'] + df['NIR'])) + (df['NIR'] / (df['NIR'] - df['Red']) + df['Green'] / (df['Green'] + df['SWIR1']))
    df['NDCCI'] = (df['NIR'] - df['Green']) / (df['NIR'] + df['Green'])

    # Vegetation indices
    df['NDVI'] = (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])
    df['SAVI'] = (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'] + 0.5) * (1 + 0.5)
    df['NDMI'] = (df['NIR'] - df['SWIR1']) / (df['NIR'] + df['SWIR1'])

    # Built-up area indices
    df['DBI'] = (df['Blue'] - df['TEMP1']) / (df['Blue'] + df['TEMP1']) - (df['NIR'] - df['Red']) / (df['NIR'] + df['Red'])
    df['NBAI'] = ((df['SWIR2'] - df['SWIR1']) / df['Green']) / ((df['SWIR2'] + df['SWIR1']) / df['Green'])
    df['NDBI'] = (df['SWIR1'] - df['NIR']) / (df['SWIR1'] + df['NIR'])

    indices = ['MBI', 'BSI', 'NDSI1', 'NDSI2', 'BI', 'DBSI', 'BAEI', 'BUI', 'NBI', 'BRBA', 'IBI', 'NDCCI', 'NDVI', 'SAVI', 'NDMI', 'DBI', 'NBAI', 'NDBI']

    for feature in list(indices + band_names):
        df[f'{feature}_mean'] = df[f'{feature}'].apply(lambda x: extract_features(x)['mean'])
        df[f'{feature}_std'] = df[f'{feature}'].apply(lambda x: extract_features(x)['std'])
        # df[f'{feature}_skewness'] = df[f'{feature}'].apply(lambda x: extract_features(x)['skewness'])
        # df[f'{feature}_kurtosis'] = df[f'{feature}'].apply(lambda x: extract_features(x)['kurtosis'])
        df[f'{feature}_percentile_25'] = df[f'{feature}'].apply(lambda x: extract_features(x)['percentile_25'])
        df[f'{feature}_percentile_50'] = df[f'{feature}'].apply(lambda x: extract_features(x)['percentile_50'])
        df[f'{feature}_percentile_75'] = df[f'{feature}'].apply(lambda x: extract_features(x)['percentile_75'])

    df = df.drop(band_names + indices, axis=1)
    df = df.drop('Image', axis=1)
    
    return df

In [16]:
df1 = calculate_indices(df1, train=True)

/tmp/ipykernel_107/2623392933.py:66: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{feature}_percentile_25'] = df[f'{feature}'].apply(lambda x: extract_features(x)['percentile_25'])
/tmp/ipykernel_107/2623392933.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{feature}_percentile_50'] = df[f'{feature}'].apply(lambda x: extract_features(x)['percentile_50'])
/tmp/ipykernel_107/2623392933.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

## Training XGBoost

In [17]:
def xgboost_plst():
    param = {}
    param['max_depth']= 7  # depth of tree
    param['learning_rate'] = 0.1     
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'
    return param

In [27]:
# def predict(_mean, _std):
#     # YOUR CODE HERE
#     #raise NotImplementedError()
    
#     pred_wo_abstention=(2*(_mean>0))-1
#     pred_with_abstention=copy(pred_wo_abstention)
#     pred_with_abstention[_std>abs(_mean)]=0
            
#     return pred_wo_abstention, pred_with_abstention

In [18]:
def calc_scores(styled_logs,title=None,normalize=True, check=False):

    for st_log in styled_logs:
        log = st_log['log']
        
        #find normalization constants
        if normalize:
            all_pred=[]
            for i in range(len(log)):
                X = log[i]
                y_pred = X['y_pred']
                all_pred.append(y_pred)
            all_pred = np.concatenate(all_pred)
            _mean = np.mean(all_pred)
            _std = np.std(all_pred)
        else:
            _mean=0
            _std=1

    if title:
        return _mean,_std

In [19]:
def train(df_train):
    
    X = df_train.drop('label', axis=1)
    y = df_train['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    log = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=100, \
                                           bootstrap_size=len(X_train))
    
    styled_logs=[
        {   'log':log
        }
    ]
    
    return styled_logs

In [20]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, bootstrap_size, \
                   num_round=200, plst=xgboost_plst()):
    
    pred_matrix = np.empty((n_bootstrap, len(X_test)))
    
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    log = []

    for i in range(n_bootstrap):

        boot_idx = np.random.choice(X_train.index, size=bootstrap_size, replace=True)
        X_sample, y_sample = X_train.loc[boot_idx], y_train.loc[boot_idx]

        dtrain = xgb.DMatrix(X_sample, label=y_sample)

        evallist = [(dtrain, 'train'), (dtest, 'eval')]

        plst=xgboost_plst()

        bst = xgb.train(plst, dtrain, num_boost_round=num_round, evals=evallist, verbose_eval=False)

        y_pred = bst.predict(dtest,output_margin=True, ntree_limit=bst.best_ntree_limit)

        y_pred = np.round(y_pred/(np.max(y_pred) - np.min(y_pred)),2)
        
        margin_scores = y_pred
        sorted_scores = np.sort(margin_scores)
        filtered_scores = sorted_scores[int(0.1 * len(sorted_scores)):int(0.9 * len(sorted_scores))+1]

        log.append({
            'i':i,
            'bst':bst,
            'y_pred': filtered_scores
        })

    return log

In [21]:
styled_logs = train(df1)

_mean,_std = calc_scores(styled_logs,title='All')

os.makedirs('data', exist_ok=True)
pickle_file='data/model.pkl'

Dump={'styled_logs':styled_logs,
      'mean':_mean,
      'std':_std}
pkl.dump(Dump,open(pickle_file,'wb'))
print('picklefile=',pickle_file)

/home/ans037/.local/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
/home/ans037/.local/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
/home/ans037/.local/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
/home/ans037/.local/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
/home/ans037/.local/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
/home/ans037/.local/lib/python3.9/site-packages/xgboost/core.py:122: UserWarning: ntree_limit is deprecated, use `iteration_range`

picklefile= data/model.pkl
